## Cuestionario

In [1]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

In [3]:
import pandas as pd      # para trabajar con archivos de datos csv, excel, etc: https://pandas.pydata.org/docs/getting_started/tutorials.html
import chardet           # para detectar la codificación de texto en archivos
import numpy as np

from sklearn import preprocessing
from matplotlib import pyplot as plt
import seaborn as sns

# importa neurona lineal de Laura en la carpeta Colab definida con FUENTES_DIR
from ClassNeuronaLineal import NeuronaLineal

nombre_archivo = DATOS_DIR + 'Frutastrain.csv' # archivo de hojas

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, delimiter=',' , nrows=None) # uso ';' por como esta organizado el dataframe

df.head()



,Diametro,Color,Clase
0,10,200,Naranja
1,20,30,Melon
2,8,150,Naranja
3,26,30,Melon
4,7,170,Naranja


In [4]:
#verifico las primeras y ultimas filas (solo para ver si se leyó bien el csv) --> tiene que haber 16 filas
print(df.head())
print(df.tail())

   Diametro  Color    Clase
0        10    200  Naranja
1        20     30    Melon
2         8    150  Naranja
3        26     30    Melon
4         7    170  Naranja
    Diametro  Color    Clase
11        23    190    Melon
12        24    250  Naranja
13        15     31    Melon
14        15    250  Naranja
15        19     31    Melon


In [74]:
#calculo media y desvio:
MEDIACOLOR= df['Color'].mean()
DESVCOLOR= df['Color'].std()
MEDIAD= df['Diametro'].mean()
DESVD= df['Diametro'].std()


#Normalizo:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Seleccionamos solo las columnas numéricas
columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns

# Aplicamos el escalador a esas columnas
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

print(df)

    Diametro     Color    Clase
0  -1.504629  0.784611  Naranja
1   0.239868 -1.402009    Melon
2  -1.853529  0.141487  Naranja
3   1.286567 -1.402009    Melon
4  -2.027978  0.398737  Naranja
5   0.937667 -1.376284    Melon
6   0.239868  0.398737  Naranja
7   0.414318  0.270112    Melon
8   0.414318  0.527361  Naranja
9   0.763218  0.270112    Melon
10  0.588768  0.655986  Naranja
11  0.763218  0.655986    Melon
12  0.937667  1.427734  Naranja
13 -0.632380 -1.389147    Melon
14 -0.632380  1.427734  Naranja
15  0.065419 -1.389147    Melon


#### Para analizar esto, primero armo la funcion tanh con los pesos: W(diámetro) = 2.65, W(color) = -2.5 y b = 0.03.

In [21]:
import numpy as np

# PESOS Y BIAS
W_diametro = 2.65
W_color = -2.5
b = 0.03



# Definir una función tanh
def tanh_activation(diametro, color):

    # Calcular la entrada neta (combinación lineal de las características)
    x = ( W_diametro * diametro ) + ( W_color * color ) + b
    
    print('neta: ' , x)

    # Aplicar la función de activación tanh
    tanh_output = (2.0 / (1+np.exp(-2*x)) - 1)

    return tanh_output 

#### Evaluo las filas del df en tanh


In [24]:
cantIndefinidos = 0
for i, row in df.iterrows():
    diametro = row['Diametro']
    color = row['Color']
    prediccion = tanh_activation(diametro, color)
    
    if (-0.8 < prediccion < 0.8):
      cantIndefinidos = cantIndefinidos + 1
        
    print(f"Fila {i} -> Diámetro: {diametro}, Color: {color} -> Predicción: {prediccion:.2f}")
    print()
print('cantidad de predicciones indefinidas ', cantIndefinidos)

neta:  -5.9187934270357125
Fila 0 -> Diámetro: -1.5046290911232887, Color: 0.7846105342235993 -> Predicción: -1.00

neta:  4.170673743910691
Fila 1 -> Diámetro: 0.2398684058312489, Color: -1.4020089873831527 -> Predicción: 1.00

neta:  -5.235568628651947
Fila 2 -> Diámetro: -1.853528590514196, Color: 0.14148714551573102 -> Predicción: -1.00

neta:  6.944424764068406
Fila 3 -> Diámetro: 1.2865669040039713, Color: -1.4020089873831527 -> Predicción: 1.00

neta:  -6.340983854052767
Fila 4 -> Diámetro: -2.02797834020965, Color: 0.3987365009988783 -> Predicción: -1.00

neta:  5.955528751811715
Fila 5 -> Diámetro: 0.937667404613064, Color: -1.376284051834838 -> Predicción: 1.00

neta:  -0.33118997704438613
Fila 6 -> Diámetro: 0.2398684058312489, Color: 0.3987365009988783 -> Predicción: -0.32

neta:  0.45266355400250036
Fila 7 -> Diámetro: 0.41431815552670265, Color: 0.27011182325730465 -> Predicción: 0.42

neta:  -0.1904598347053679
Fila 8 -> Diámetro: 0.41431815552670265, Color: 0.5273611787

In [46]:
from ClassNeuronaGral import NeuronaGradiente
modelo_sigmoid = NeuronaGradiente(alpha=0.01, n_iter=100, FUN='sigmoid', COSTO='EC_binaria')

In [102]:
modelo_tanh = NeuronaGradiente(alpha=0.01, n_iter=100, FUN='tanh', COSTO='EC_binaria')

# Asignar pesos y bias directamente desde fuera de la clase
NeuronaGradiente.w_ = np.array([2.65, -2.5])  # Asignar pesos manualmente
NeuronaGradiente.b_ = 0.03  # Asignar bias manualmente

# Datos de entrada (una muestra con dos características)
X1 = np.array([[(14 - MEDIAD)/DESVD, (140 - MEDIACOLOR)/DESVCOLOR]]) 
X2 = np.array([[(16 - MEDIAD)/DESVD, (79 - MEDIACOLOR)/DESVCOLOR]]) 

# calculos:
# Calcular la entrada neta (combinación lineal de las características)
n =  (2.65 * ((16 - MEDIAD)/DESVD )) + ( -2.5 * ((79 - MEDIACOLOR)/DESVCOLOR) ) + 0.03
    
print('neta: ' , n)

# Aplicar la función de activación tanh
tanh_output = (2.0 / (1+np.exp(-2*n)) - 1)
print('salida' , tanh_output)


# Predicción
#prediccion = modelo_tanh.predict(X2)
#print(prediccion)

neta:  -150.14492924919261
salida -1.0
